# Práctica 2a 
Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en
tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de
escala.

In [1]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
h, w = image.shape[:2]

cv.namedWindow("Transformaciones")

def nothing(x):
    pass


cv.createTrackbar("Tx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)  
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("Usa los sliders para transformar la imagen.")
print("Pulsa ESC para salir.")

while True:
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w//2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h//2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    # Traslación
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    img_T = cv.warpAffine(image, T, (w, h))

    # Rotación
    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])

    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([M_rotate, [0, 0, 1]])
    M3 = np.vstack([M_translate2, [0, 0, 1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w, h))

    # Escalado
    if uniforme:
        s = sx
        S = np.float32([[s, 0, 0],
                        [0, s, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])

    img_final = cv.warpAffine(img_R, S, (w, h))

    img_display = img_final.copy()
    
    cv.circle(img_display, (cx, cy), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx + 10, cy - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    tx_point, ty_point = w//2 + tx, h//2 + ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point + 10, ty_point - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


Usa los sliders para transformar la imagen.
Pulsa ESC para salir.


In [3]:
import cv2 as cv
import numpy as np

# Cargar imagen
image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
h, w = image.shape[:2]

# Variables globales para el centro de rotación
cx = float(w // 2)
cy = float(h // 2)
M_display = np.array([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0]], dtype=np.float32)

# Crear ventana
cv.namedWindow("Transformaciones")

def nothing(x):
    pass

# Crear trackbars
cv.createTrackbar("Tx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)  
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

# Callback del ratón para cambiar centro de rotación
def mouse_callback(event, x, y, flags, param):
    global cx, cy, M_display
    
    if event == cv.EVENT_LBUTTONDOWN:
        # Convertir punto clicado a coordenadas originales
        M_inv = cv.invertAffineTransform(M_display)
        src_x = M_inv[0, 0] * x + M_inv[0, 1] * y + M_inv[0, 2]
        src_y = M_inv[1, 0] * x + M_inv[1, 1] * y + M_inv[1, 2]
        
        # Actualizar centro
        cx, cy = float(src_x), float(src_y)
        
        # Ajustar traslación para mantener imagen fija
        tx_new = int(round(x - cx))
        ty_new = int(round(y - cy))
        
        cv.setTrackbarPos("Tx", "Transformaciones", 
                         np.clip(tx_new + w//2, 0, w))
        cv.setTrackbarPos("Ty", "Transformaciones", 
                         np.clip(ty_new + h//2, 0, h))

cv.setMouseCallback("Transformaciones", mouse_callback)

print("Usa los sliders para transformar la imagen.")
print("Haz CLIC para cambiar el centro de rotación.")
print("Pulsa ESC para salir.")

while True:
    # Leer valores de trackbars
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w//2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h//2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    if uniforme:
        sy = sx

    # Construir matriz compuesta
    rad = np.deg2rad(angle)
    c, s = np.cos(rad), np.sin(rad)

    # 1. Trasladar centro al origen
    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    
    # 2. Escalar
    if uniforme:
        S = np.float32([[sx, 0, 0],
                        [0, sx, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])
    
    # 3. Rotar
    M_rotate = np.float32([[c, -s, 0],
                          [s,  c, 0]])
    
    # 4. Trasladar centro de vuelta
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])
    
    # 5. Trasladar según trackbar
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])

    # Combinar todas las transformaciones
    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([S, [0, 0, 1]])
    M3 = np.vstack([M_rotate, [0, 0, 1]])
    M4 = np.vstack([M_translate2, [0, 0, 1]])
    M5 = np.vstack([T, [0, 0, 1]])
    
    M_total = M5 @ M4 @ M3 @ M2 @ M1
    M_final = M_total[:2]
    
    # Guardar matriz para el callback
    M_display = M_final.copy()

    # Aplicar transformación
    img_final = cv.warpAffine(image, M_final, (w, h))

    # Crear copia para visualización
    img_display = img_final.copy()
    
    # Calcular posición del centro en la imagen transformada
    px = int(round(M_final[0, 0]*cx + M_final[0, 1]*cy + M_final[0, 2]))
    py = int(round(M_final[1, 0]*cx + M_final[1, 1]*cy + M_final[1, 2]))
    
    # Dibujar centro de rotación (rojo)
    cv.circle(img_display, (px, py), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({int(cx)},{int(cy)})", (px + 10, py - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    # Dibujar punto de traslación (verde)
    tx_point = w//2 + tx
    ty_point = h//2 + ty
    if 0 <= tx_point < w and 0 <= ty_point < h:
        cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
        cv.putText(img_display, f"T({tx_point},{ty_point})", 
                   (tx_point + 10, ty_point - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()

Usa los sliders para transformar la imagen.
Haz CLIC para cambiar el centro de rotación.
Pulsa ESC para salir.
